In [1]:
import os
import folderstats
import matplotlib.pyplot as plt
import squarify
import numpy as np
import datetime

#parent folder of images (children should be categories)
folder = '/raid/arnold/clouds_detection/ALL/'
filepaths = [os.path.join(folder, f) for f in os.listdir(folder)]

#print(os.stat(folder))
df = folderstats.folderstats(folder, ignore_hidden=True)
df.head()
#only look at folders
df_folder = df[df['folder']]
df_folder[['name', 'num_files']]

,name,num_files
4777,rain,4777.0
14625,overcast,9847.0
21260,clear,6634.0
26405,few,5144.0
38141,scattered,11735.0
44080,night,5938.0
44805,snow,724.0
44806,,44799.0


In [2]:
#find folders
#df[df['folder']]

#drop broken rows
#df_folder = df_folder.drop(index=44807, axis=0)
# df_folder.head()

In [3]:
#print number of files for each category 
cloud_num = df_folder[['name', 'num_files']]
cloud_num = cloud_num.set_index('name')
cloud_num

,num_files
name,
rain,4777.0
overcast,9847.0
clear,6634.0
few,5144.0
scattered,11735.0
night,5938.0
snow,724.0
,44799.0


In [5]:

#drop all folders
df.drop(df.loc[df['folder']==True].index, inplace=True)
df.head() #uncomment to view

,id,path,name,extension,size,atime,mtime,ctime,folder,num_files,depth,parent,uid
0,3,/raid/arnold/clouds_detection/ALL/rain/2019061...,20190610T170526_BKLN,jpg,88606,2021-06-10 13:47:18,2020-10-20 15:49:12,2021-02-11 18:22:30,False,NaN,1,2,65534
1,4,/raid/arnold/clouds_detection/ALL/rain/2019050...,20190505T171527_BKLN,jpg,99646,2021-06-10 13:53:18,2020-10-20 15:40:13,2021-02-11 18:22:30,False,NaN,1,2,65534
2,5,/raid/arnold/clouds_detection/ALL/rain/2019062...,20190621T153526_BKLN,jpg,81607,2021-06-10 13:51:08,2020-10-20 15:48:04,2021-02-11 18:22:30,False,NaN,1,2,65534
3,6,/raid/arnold/clouds_detection/ALL/rain/2019062...,20190620T145027_BKLN,jpg,80033,2021-06-10 13:49:58,2020-10-20 15:49:37,2021-02-11 18:22:30,False,NaN,1,2,65534
4,7,/raid/arnold/clouds_detection/ALL/rain/2019051...,20190514T234026_BKLN,jpg,79198,2021-06-10 14:11:23,2020-10-20 15:41:00,2021-02-11 18:22:30,False,NaN,1,2,65534


In [ ]:
**** need to edit to get correct classification since new path and change from cloud type to class or class type
#get cloud type and make a new column for it
# cloud_type = df["path"].str.split("raindrop-detection-cnn/", n=1, expand = True)
# cloud_type.head()
#not working ffor some reason
# for i in range(0, len(df)): 
#     df.iloc[i].path = df.iloc[i].path[17:-1] 

# df['path'] = df['path'].str.slice(17, -1)     
# df.head()
cloud_type = df['path'].str.slice(41, -4)  
df['path'] = cloud_type
# new data frame with split value columns to seperate time from name
new_split = df["path"].str.split("/", n = 1, expand = True) 
df['cloud'] = new_split[0]
# df.head() #uncomment to view

In [ ]:
# new data frame with split value columns to seperate time from name
new_split = df["path"].str.split("/", n = 1, expand = True) 
df['cloud'] = new_split[0]
# df.head() #uncomment to view

In [ ]:
# dropping null value columns to avoid errors 
# data.dropna(inplace = True) 
  
# new data frame with split value columns to seperate time from name
new = df["name"].str.split("_", n = 1, expand = True) 
# print(new)
# making separate date column from new data frame and merge into orignal df
df["date"]= new[0] 
  
# making separate name column from new data frame and merge into orignal df
df["site_name"]= new[1] 
  
# Dropping old Name columns 
df.drop(columns =["name"], inplace = True) 
  
# df display 
# df #uncomment to view

In [ ]:
import pandas as pd

# converting the string to datetime format 
# df['Dates'] = pd.to_datetime(df['date'], format='%Y%m%dT%H%M%S') 
df['Dates'] = pd.to_datetime(df['date'], format='%Y%m%dT%H%M%S') 
# df.head() #uncomment to view

In [ ]:
#create a new column with year of date field
df['year'] = pd.DatetimeIndex(df['date']).year
#create a new column with month of date field
df['month'] = pd.DatetimeIndex(df['date']).month
#since we have datetime obj, can extract the day/month/year using the to_period function
#where 'D', 'M', 'Y' are inputs
df['month_year'] = pd.to_datetime(df['date']).dt.to_period('M')
# df.head() #uncomment to view

In [ ]:
#drop cols that are not needed
# Dropping old Name columns 
df.drop(columns =["id", "extension", "atime", "mtime", "ctime", "folder", "depth", "parent", "uid"], inplace = True) 
df.head() #now we have the cleaned up df with the information we want and can visualize 

In [ ]:
#make month_year the index
df = df.set_index('month_year')
df.head()


In [ ]:
cloud_num.loc['rain']

In [ ]:
#add num files from cloud_num into df
#works for single condition
# df['num_files'] = np.where(df['cloud']== 'rain', cloud_num.loc['rain'], False)
df['num_files'] = np.where(df['cloud']== 'rain', cloud_num.loc['rain'], 
                           np.where(df['cloud']== 'overcast', cloud_num.loc['overcast'], 
                                    np.where(df['cloud']== 'clear', cloud_num.loc['clear'], 
                                             np.where(df['cloud']== 'few', cloud_num.loc['few'], 
                                                      np.where(df['cloud']== 'scattered', cloud_num.loc['scattered'], 
                                                               np.where(df['cloud']== 'night', cloud_num.loc['night'], 
                                                                        np.where(df['cloud']== 'snow', cloud_num.loc['snow'], False)))))))

In [ ]:
df

In [ ]:
#show distribution by month
with plt.style.context('ggplot'):
    df['month'].value_counts().plot(
        kind='bar', color='C1', title='Distribution by month');

In [ ]:
#show distribution by month
with plt.style.context('ggplot'):
    df['cloud'].value_counts().plot(
        kind='bar', color='C1', title='Overcast Distribution by month');

In [ ]:
#plotting and saving the plots
df.groupby(['month','cloud']).count()['num_files'].unstack().plot( kind='bar', )

# the plot gets saved to 'output.png'
# plt.savefig('output.png')

In [ ]:
# df[["year", "month", "cloud", "site_name"]].plot(x="year", kind="bar")
fig, ax = plt.subplots(figsize=(15,7))
df.groupby(['month']).count()['cloud'].plot(ax=ax)

In [ ]:
# plot data
fig, ax = plt.subplots(figsize=(15,7))
# use unstack()
df.groupby(['month','cloud']).count()['num_files'].unstack().plot(ax=ax)

In [ ]:
df_folder.plot(x='name', y='num_files', kind='bar') 


In [ ]:
#show distribution by file size
with plt.style.context('ggplot'):
    # Group by extension and sum all sizes for each extension 
#     extension_sizes = df.groupby('month')['cloud'].sum()
    extension_sizes = df.groupby('cloud')['month'].sum()
    # Sort elements by size
    extension_sizes = extension_sizes.sort_values(ascending=False)
    
    extension_sizes.plot(
        kind='bar', color='C1', title='Overcast Distribution by month');

In [ ]:
#Tree map

# Group by extension and sum all sizes for each extension
extension_sizes = df.groupby('cloud')['month'].sum()
# Sort elements by size
extension_sizes = extension_sizes.sort_values(ascending=False)

squarify.plot(sizes=extension_sizes.values, label=extension_sizes.index.values)
plt.title('Overcast Treemap by month')
plt.axis('off');

In [ ]:
with plt.style.context('ggplot'):
    # Filter the data set to only folders
    df_folders = df[df['folder']]
    # Set the name to be the index (so we can use it as a label later)
    df_folders.set_index('name', inplace=True)
    # Sort the folders by size
    df_folders = df_folders.sort_values(by='num_files', ascending=False)
    
    # Show the size of the largest 50 folders as a bar plot
    df_sizes['num_files'][:50].plot(kind='bar', color='C0', title='Category # files');

In [ ]:
with plt.style.context('ggplot'):
    y, bins = np.histogram(df['size'], bins=10000)
    plt.loglog(bins[:-1], y, '.');
    plt.ylabel('Size')
    plt.xlabel('Rank')


In [ ]:
import networkx as nx

# Sort the index
df_sorted = df.sort_values(by='id')

G = nx.Graph()
for i, row in df_sorted.iterrows():
    if row.parent:
        G.add_edge(row.id, row.parent)
    
# Print some additional information
#nodes (files and folders)
print(nx.info(G))

In [ ]:
from networkx.drawing.nx_pydot import graphviz_layout

pos_dot = graphviz_layout(G, prog='dot')

fig = plt.figure(figsize=(16, 8))
nodes = nx.draw_networkx_nodes(G, pos_dot, node_size=2, node_color='C0')
edges = nx.draw_networkx_edges(G, pos_dot, edge_color='C0', width=0.5)
plt.axis('off');

In [ ]:
pos_twopi = graphviz_layout(G, prog='twopi', root=1)

fig = plt.figure(figsize=(14, 14))
nodes = nx.draw_networkx_nodes(G, pos_twopi, node_size=2, node_color='C0')
edges = nx.draw_networkx_edges(G, pos_twopi, edge_color='C0', width=0.5)
plt.axis('off')
plt.axis('equal');